In [43]:
import altair as alt
import pandas as pd
import numpy as np
# norms
# mean sj: (1,1,0,0,0,0,1,1)
# nice sj: (1,1,0,1,0,1,1,1)

ebnorms_b = ["(1,1,0,0,0,0,1,1)", "(1,1,0,1,0,1,1,1)", "(1,1,1,0,1,0,1,1)", "(0,0,0,0,1,1,1,1)", "(0,1,0,1,1,1,1,1)",
          "(1,1,0,0,1,1,1,1)", "(1,1,0,1,1,1,1,1)", "(0,0,0,0,0,0,1,1)", "(0,1,0,1,0,1,1,1)"]

ebnorms_n = ["SJ normal", "SJ Nice", "SJ Mean", "IS normal", "IS nice", "SS normal", "SS nice", "SH normal", "SH normal"]

norms_simple_b = ["(1,0,0,1)", "(1,0,1,1)", "(1,0,0,0)", "(1,1,0,0)"]

norms_simple_n = ["SJ", "SS", "SH", "IS"]

columns = ["EBNorm", "Norm", "z", "g", "mu", "chi", "eps", "gamma", "pdx", "acr", "B", "G", "AllD", "pDisc", "Disc", "AllC", "mean", "nice"]

data: pd.DataFrame = pd.read_csv("results.txt", sep="\t", header=None, names=columns, index_col=False)

data.eps = data.eps / data.z

data.dropna()
data.EBNorm = data.EBNorm.replace(ebnorms_b, ebnorms_n)
data.Norm = data.Norm.replace(norms_simple_b, norms_simple_n)

def filter(frame: pd.DataFrame, feature: str, condition):
    return frame[frame[feature]==condition]
    

In [44]:
data

,EBNorm,Norm,z,g,mu,chi,eps,gamma,pdx,acr,B,G,AllD,pDisc,Disc,AllC,mean,nice
0,SJ normal,SJ,40.0,4000.0,0.1,0.1,0.00250,0.50,True,64.562907,0.475,0.525,1.0,0.0,0.0,0.0,0.0,1.0
1,SJ Nice,SJ,40.0,4000.0,0.1,0.1,0.00250,0.50,True,76.796894,1.000,0.000,0.0,0.0,1.0,0.0,1.0,0.0
2,SJ Mean,SJ,40.0,4000.0,0.1,0.1,0.00250,0.50,True,85.064990,0.450,0.550,1.0,0.0,0.0,0.0,0.0,1.0
3,SJ normal,SJ,50.0,4000.0,0.1,0.1,0.00002,0.00,True,96.137431,0.000,1.000,0.0,1.0,0.0,0.0,0.0,1.0
4,SJ normal,SJ,50.0,4000.0,0.1,0.1,0.00002,0.25,True,99.378976,0.000,1.000,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9707,SJ Mean,SJ,50.0,4000.0,0.1,0.1,0.20000,0.60,True,66.321119,0.900,0.100,0.0,0.0,1.0,0.0,0.0,1.0
9708,SJ Mean,SJ,50.0,4000.0,0.1,0.1,0.20000,0.70,True,49.301593,0.760,0.240,0.0,0.0,1.0,0.0,0.0,1.0
9709,SJ Mean,SJ,50.0,4000.0,0.1,0.1,0.20000,0.80,True,66.285903,0.760,0.240,0.0,0.0,1.0,0.0,0.0,1.0
9710,SJ Mean,SJ,50.0,4000.0,0.1,0.1,0.20000,0.90,True,66.302454,0.840,0.160,0.0,0.0,1.0,0.0,0.0,1.0


In [41]:
chosen_norm: str = "SJ normal"
filtered = data[data.EBNorm == chosen_norm]
filtered = filter(filtered, "z", 50)

new_cols = ["gamma", "eps", "acr", "g_reps", "nice_eps", "Disc", "AllC"]
new_df = pd.DataFrame(columns=new_cols)

gammas = [round(a, 2) for a in np.arange(0, 1.01, 0.1)]

for gamma in gammas:
    for epsilon in sorted(filtered.eps.unique()):
        super_filtered = filtered[(filtered.gamma == gamma) & (filtered.eps == epsilon)]
        #print(super_filtered.shape)
        mean_acr: float = super_filtered.acr.mean()
        mean_greps: float = super_filtered.G.mean()
        mean_niceeps: float = super_filtered.nice.mean()
        #print(gamma, epsilon, mean_acr)
        mean_coops: float = super_filtered.AllC.mean() + super_filtered.Disc.mean()
        mean_defects: float = super_filtered.AllD.mean() + super_filtered.pDisc.mean()
        new_df.loc[len(new_df)] = [gamma, epsilon, mean_acr, mean_greps, mean_niceeps, mean_coops, mean_defects]
            

feature: str = "acr" + ":Q"
            
heatmap = alt.Chart(new_df).mark_rect().encode(
    x='eps:O',
    y=alt.Y('gamma:O', sort=alt.EncodingSortField('gamma', order='descending')),
    color=feature
).properties(
    height=400,
    width=400,
    title = chosen_norm
)

text = heatmap.mark_text().encode(
    text=alt.Text(feature, format=".2f"),
    color=alt.value('black')
)

#heatmap+text

In [42]:
heatmap + text



alt.LayerChart(...)

In [9]:
sorted(filtered.eps.unique())

[2e-05, 0.0002, 0.002, 0.02, 0.2]

In [10]:
chosen_norm: str = "SJ Mean"
filtered = data[data.EBNorm == chosen_norm]
filtered = filter(filtered, "z", 50)

new_cols = ["gamma", "eps", "acr"]
new_df = pd.DataFrame(columns=new_cols)

for gamma in sorted(filtered.gamma.unique()):
    for epsilon in sorted(filtered.eps.unique()):
        super_filtered = filtered[(filtered.gamma == gamma) & (filtered.eps == epsilon)]
        #print(super_filtered.shape)
        mean_acr: float = super_filtered.Disc.mean()
        #print(gamma, epsilon, mean_acr)
        new_df.loc[len(new_df)] = [gamma, epsilon, mean_acr]
        
#print(new_df)
        
heatmap = alt.Chart(new_df).mark_rect().encode(
    x='eps:O',
    y='gamma:O',
    color='acr:Q'
).properties(
    height=400,
    width=400,
    title = chosen_norm
)

text = heatmap.mark_text().encode(
    text=alt.Text('acr:Q', format=".2f"),
    color=alt.value('black')
)

heatmap + text


alt.LayerChart(...)